In [90]:
import six
import copy
import pandas as pd
from pptx import Presentation
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
from pptx.util import Pt

In [91]:
def make_cert_no_format(prs):
    slide = prs.slides[0]
    slide.shapes.element.remove(slide.shapes[0].element)
    new = slide.shapes.add_shape(1, 600000, 700000, 4000000, 250000)
    new.text = "수료증 번호"
    new.fill.solid()
    new.fill.fore_color.rgb = RGBColor(0xFF, 0xFF, 0xFF)
    new.line.color.rgb = RGBColor(0xFF, 0xFF, 0xFF)

def duplicate_slide(prs, org_slide):
    copied_slide = prs.slides.add_slide(org_slide.slide_layout)
    for shape in org_slide.shapes:
        org_elment = shape.element
        new_element = copy.deepcopy(org_elment)
        copied_slide.shapes._spTree.insert_element_before(new_element, "p:extLst")
    for _, value in six.iteritems(org_slide.part.rels):
        if "notesSlide" not in value.reltype:
            copied_slide.part.rels.add_relationship(
                value.reltype,
                value._target,
                value.rId
            )
    return copied_slide

def delete_slide(prs, slide_idx):
    slides_lst = list(prs.slides._sldIdLst)
    prs.slides._sldIdLst.remove(slides_lst[slide_idx])
    
def delete_layout_element(prs, slide_no):
    slide_layout = prs.slides[slide_no].slide_layout
    slide_layout.shapes.element.remove(slide_layout.shapes[0].element)
    slide_layout.shapes.element.remove(slide_layout.shapes[0].element)

def make_slide(df, prs):
    delete_layout_element(prs, 0)
    delete_layout_element(prs, 1)
    
    df_cert = df[(df.구분=="수료")]
    slide_cert = prs.slides[0]
    slide_prize_1 = prs.slides[1]
    slide_prize_2 = prs.slides[2]
    
    cert_no = 0
    for item in df_cert.iterrows():
        cert_no += 1
        new_slide = duplicate_slide(prs, slide_cert)
        for shape in new_slide.shapes:
            if shape.has_text_frame and shape.text_frame.text == "수료증 번호":
                shape.text_frame.text = "JuniorAI-21-3-{}".format(cert_no)
                shape.text_frame.paragraphs[0].alignment = PP_ALIGN.LEFT
                shape.text_frame.paragraphs[0].font.name = "나눔명조"
                shape.text_frame.paragraphs[0].font.bold = False
                shape.text_frame.paragraphs[0].font.size = Pt(10)
                shape.text_frame.paragraphs[0].font.color.rgb = RGBColor(0x00, 0x00, 0x00)
            if shape.has_table:
                shape.table.cell(0,2).text = item[1][1]
                name_cell = shape.table.cell(0,2).text_frame.paragraphs[0]
                name_cell.alignment = PP_ALIGN.LEFT
                name_cell.font.name = "나눔스퀘어"
                name_cell.font.bold = False
                name_cell.font.size = Pt(16)
                name_cell.font.color.rgb = RGBColor(0x00, 0x00, 0x00)
                
    df_prize = df[(df.구분=="표어") | (df.구분=="우수")]
    for item in df_prize.iterrows():
        if item[1][0] == "표어":
            new_slide = duplicate_slide(prs, slide_prize_1)
        elif item[1][0] == "우수":
            new_slide = duplicate_slide(prs, slide_prize_2)
        for shape in new_slide.shapes:
            if shape.has_text_frame and shape.text_frame.text == "이름 : 김가연":
                shape.text_frame.text = "이름 : " + item[1][1]
                shape.text_frame.paragraphs[0].alignment = PP_ALIGN.LEFT
                shape.text_frame.paragraphs[0].font.name = "나눔명조 ExtraBold"
                shape.text_frame.paragraphs[0].font.bold = True
                shape.text_frame.paragraphs[0].font.size = Pt(18)
                shape.text_frame.paragraphs[0].font.color.rgb = RGBColor(0x00, 0x00, 0x00)    

    delete_slide(prs, 0)
    delete_slide(prs, 0)
    delete_slide(prs, 0)

In [92]:
df = pd.read_clipboard()
prs = Presentation(r"C:\Users\poscouser\Desktop\수료증 양식.pptx")
make_cert_no_format(prs)
make_slide(df, prs)
prs.save(r"C:\Users\poscouser\Desktop\수료증 작업결과.pptx")